In [ ]:
### PLOTLY FUNCTIONS
### Authors: JONATHAN CHAN 

###MOST RECENT UPDATE:  
##2020 JUNE 1, 2:26PM
#Added function for combined chart

#TO DO:
#test with other financial indicators - real data

In [33]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from datetime import datetime
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import random
from plotly.subplots import make_subplots

In [107]:
def plot_combined_graph(indicator_df, senti_df, indicator_name="y-axis label", title="Default Title", add_rangeslide=True): 
    """
    returns a Plotly graph given a dataframes containing financial indicator data and 
    a dataframe containing sentiment values 
    
    INPUT:
    
    indicator_df: dataframe containing a "date" column of datetimes and a "values" column of float values
    senti_df: dataframe containing a "final_sentiment" column of float values
    
    indicator_name: string label for y axis (what is tracked in indicator_df["values"])
    title: string of title 
    add_rangeslide: boolean of displaying
    
    
    """
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    indi_y = indicator_df["values"].astype('float64')
    dates = indicator_df["dates"]
    senti_y = senti_df["final_sentiment"].astype('float64')
    
    #check that 
    assert(len(dates) == len(senti_y))
    
    #Add indicator area visualization - set boundaries, add to fig object, update axis
    indi_axis_min = min(indi_y) - 0.1
    indi_axis_max = max(indi_y) + 0.1

    fig.add_trace(
        go.Scatter(x=dates, 
                   y=indi_y, 
                    name=indicator_name,
                    fill='tonexty', 
                    mode='lines', 
                    line_color='blue'),
                    secondary_y=False)
    fig['layout']['yaxis1'].update(title=indicator_name, 
                                   range=[indi_axis_min, indi_axis_max], 
                                   autorange=False)
    
    #Add sentiment line visualization - set boundaries, add to fig object, update axis
    senti_axis_min = min(senti_y) - 1
    senti_axis_max = max(senti_y) + 1
    
    fig.add_trace(
        go.Scatter(x=dates, 
                   y=senti_y, 
                   name="test sentiment",
                  line_color='orange'),
                    secondary_y=True)
    fig['layout']['yaxis2'].update(title='Sentiment Score', 
                                   range=[senti_axis_min, senti_axis_max])
    
    #update x axis, add title, and show
    fig.update_xaxes(rangeslider_visible=add_rangeslide, title_text="Date")
    fig.update_layout(title_text=title)
    #fig.show()
    return fig

    


#### Example use below - replace parse_govt_data() function with Sirisha's function soon:

In [108]:
#REPLACE THIS CELL WITH SIRISHA's CODE
#create toy dataframe for indicator data - use outdated function for now, replace with Sirisha's dataframe code

def parse_govt_data(df, daterow_ind, valuerow_ind, datetime_format_in):
    """returns a list of datetime strings and financial indicator values from 
    dataframe of govt data
    
    assume datetime format for output datetime list = YYYY-MM-DD
    
    """
    date_row = df.iloc[daterow_ind]
    date_row = date_row[2:]
    date_list = date_row.tolist()
    datetime_list = []

    for item in date_list:
        dt = datetime.strptime(item, datetime_format_in)
        datetime_list.append(dt.strftime("%Y-%m-%d"))


    value_row = df.iloc[valuerow_ind]
    value_row = value_row[2:]
    value_list = value_row.tolist()
    
    assert (len(value_list) == len(datetime_list))

    return datetime_list, value_list

datetime_mortgage, values_mortgage = parse_govt_data(mortgage_df, 0, 2, "%B %Y")
filepath = "govt_data/mortgage_rate_jan-2019_feb-2020.csv"
mortgage_df = pd.read_csv(filepath, skiprows=5)
mortgage_df.head()

final_df = pd.DataFrame(values_mortgage, 
                        index =datetime_mortgage, 
                        columns =['values']).reset_index()

final_df["indicator"] = "GDP"
test_mrg_df = final_df.rename(columns={"index": "dates"})

test_mrg_df.head()

,dates,values,indicator
0,2019-01-01,3.84,GDP
1,2019-02-01,3.82,GDP
2,2019-03-01,3.76,GDP
3,2019-04-01,3.59,GDP
4,2019-05-01,3.45,GDP


In [109]:

#make toy sentiment data
test_senti = [random.uniform(-1, 1) for i in range(len(values_mortgage))]
test_senti_df = pd.DataFrame(test_senti, columns=["final_sentiment"])
test_senti_df.head()

,final_sentiment
0,-0.144307
1,-0.728270
2,0.863293
3,-0.694657
4,-0.827297


In [110]:
#test function  
test_indicator_name = "Interest Rate  <br> (total, funds advanced, <br> residential mortgages, insured)"
test_title = "Canadian News Sentiment - Mortgage Rate (test)"

test_fig = plot_combined_graph(test_mrg_df, test_senti_df, test_indicator_name, test_title)
test_fig.show()